# Prepare Data

In [ ]:
CREATE or REPLACE file format csvformat
  SKIP_HEADER = 1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  type = 'CSV';

CREATE or REPLACE stage call_transcripts_data_stage
  file_format = csvformat
  url = 's3://sfquickstarts/misc/call_transcripts/';

CREATE or REPLACE table CALL_TRANSCRIPTS ( 
  date_created date,
  language varchar(60),
  country varchar(60),
  product varchar(60),
  category varchar(60),
  damage_type varchar(90),
  transcript varchar
);

COPY into CALL_TRANSCRIPTS
  from @call_transcripts_data_stage;

In [ ]:
SELECT * FROM CALL_TRANSCRIPTS;

# Translate

In [ ]:
SELECT SNOWFLAKE.cortex.translate('how are you?','en_US','fr');

In [ ]:
SELECT TRANSCRIPT,SNOWFLAKE.CORTEX.TRANSLATE(TRANSCRIPT,'fr','en_XX')
FROM CALL_TRANSCRIPTS 
WHERE LANGUAGE = 'French';

# Sentiment
> Note:
>
> Score is floating point between `-1` and `1`; 
> * `-1` = most negative, 
> * `0`  = neutral
> * `1`  = positive

In [ ]:
SELECT TRANSCRIPT, SNOWFLAKE.CORTEX.SENTIMENT(TRANSCRIPT) 
FROM CALL_TRANSCRIPTS
WHERE LANGUAGE = 'English';

# Summarize

In [ ]:
SELECT TRANSCRIPT,SNOWFLAKE.CORTEX.SUMMARIZE(TRANSCRIPT) 
FROM CALL_TRANSCRIPTS
WHERE LANGUAGE = 'English' limit 1;

# Prompt Engineering

In [ ]:
SET prompt = 
'###
Summarize this transcript in less than 200 words. 
Put the product name, defect and summary in JSON format. 
###';
SELECT SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic',concat('[INST]',$prompt,transcript,'[/INST]')) AS summary
FROM call_transcripts WHERE LANGUAGE = 'English' LIMIT 1;

> **NOTE**:
> 
> Snowflake currently suppports the following LLM:
> * snowflake-arctic
> * mistral-large
> * reka-flash
> * reka-core
> * mixtral-8x7b
> * llama2-70b-chat
> * llama3-8b
> * llama3-70b
> * mistral-7b
> * gemma-7b 